# Test Matrix
Report the test matrix from the ReFrame config.

This is very `hpc-tests` specific, as it assumes tests map 1:1 to environments.

In [9]:
import sys, json, pprint
import pandas as pd

CFG_PATH = '../reframe_config.py'

def read_config(path):
    config = {}
    with open(path) as cfgf:
        exec(cfgf.read(), {}, config)
    return config['site_configuration']

matrix = {} # environment (= test) name -> ['system1:partition1', ...]
all_sys_parts = [] # ['system1:partition1', ...]

cfg = read_config(CFG_PATH)
for system in cfg['systems']:
    sys_name = system['name']
    for partition in system['partitions']:
        part_name = partition['name']
        #print(' ', part_name)
        sys_part = '%s:%s' % (sys_name, part_name)
        all_sys_parts.append(sys_part)
        for env_name in sorted(partition['environs']):
            envs = [e for e in cfg['environments'] if e['name'] == env_name]
            sys_envs = [e for e in envs if sys_part in e.get('target_systems', [])]
            assert len(sys_envs) < 2, sys_envs
            default_envs = [e for e in envs if not 'target_systems' in e]
            assert len(default_envs) < 2, default_env

            if sys_envs or default_envs:
                if env_name not in matrix:
                    matrix[env_name] = {}
                matrix[env_name][sys_part] = 'Y'

# fill in False values:
for env in matrix:
    for sys_part in all_sys_parts:
        if sys_part not in matrix[env]:
            matrix[env][sys_part] = ''

# produce dataframe for neat display:
df = pd.DataFrame(matrix)
display(df)

,castep,imb,nxnlatbw,sysinfo,gromacs,omb,openfoam,intel-hpcg,intel-hpl,cp2k,hpl
csd3:cclake-ib-icc19-impi19-ucx,Y,Y,Y,Y,,,,,,,
csd3:cclake-roce-icc19-impi19-ucx,Y,Y,,,,,,,,,
csd3:cclake-ib-gcc9-ompi3-ucx,,Y,,,Y,Y,Y,,,,
csd3:cclake-roce-gcc9-ompi3-ucx,,Y,,,Y,Y,Y,,,,
alaska:ib-gcc9-impi-verbs,,Y,,,,Y,,Y,Y,,
alaska:roce-gcc9-impi-verbs,,Y,,,,Y,,Y,Y,,
alaska:ib-gcc9-openmpi4-ucx,,Y,,Y,Y,Y,Y,,,Y,Y
alaska:roce-gcc9-openmpi4-ucx,,Y,,,Y,Y,Y,,,Y,Y
